# Comparación de Modelos Supervisados con Validación Cruzada

Notebook pensado para un público general. Cada sección explica qué se hace, por qué se hace y cómo interpretar los resultados. Caso económico: riesgo de crédito (Give Me Some Credit, Kaggle). Métrica principal: ROC AUC y partición 70/15/15 estratificada.

## Contexto y decisiones de diseño
- Objetivo: predecir morosidad seria en 2 años (SeriousDlqin2yrs).
- Datos: más de 10 predictores numéricos (edad, ingresos, utilización de crédito, atrasos, etc.).
- Decisiones: ROC AUC por desbalance, particiones estratificadas, pipelines sin fuga de información, comparación de tres modelos clásicos.
- Criterio de éxito: mayor AUC promedio en CV y coherencia con el conjunto de prueba.

## Guía de lectura
1. Diagnóstico y EDA.
2. Partición estratificada + pipeline reproducible.
3. Validación cruzada y comparación de modelos.
4. Selección y evaluación en prueba (ROC + matriz de confusión).
5. Interpretaciones automáticas + resumen exportable al README.

In [ ]:
# Importaciones y configuración
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')
plt.rcParams['figure.figsize'] = (8, 5)

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

SEED = 42
np.random.seed(SEED)
DATA_PATH = Path('data')
CSV_FILE = DATA_PATH / 'cs-training.csv'
assert CSV_FILE.exists(), 'No se encontró data/cs-training.csv'

## Dataset y objetivo
- Qué: cargar cs-training.csv, definir variable objetivo y revisar faltantes.
- Por qué: documentar el estado del dataset antes de entrenar y planear imputaciones.

In [ ]:
df = pd.read_csv(CSV_FILE)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

target = 'SeriousDlqin2yrs'
features = [c for c in df.columns if c != target]

print('Forma del dataset:', df.shape)
display(df.head())
miss = df.isna().sum().sort_values(ascending=False)
display(pd.DataFrame({'faltantes': miss[miss > 0], '%': (miss[miss > 0] / len(df) * 100).round(2)}))

## EDA: distribución e implicaciones
- Visualizamos el desbalance de la clase objetivo.
- Exploramos histogramas de variables clave para detectar escalas y outliers.
Interpretación: al haber desbalance evitamos depender de accuracy y usamos ROC AUC / PR.

In [ ]:
ax = df[target].value_counts().sort_index().plot(kind='bar', color=['#4C78A8', '#F58518'])
ax.set_xticklabels(['No morosidad seria', 'Morosidad seria'])
ax.set_ylabel('Número de observaciones')
ax.set_title('Distribución de la clase objetivo')
plt.show()

cols = ['RevolvingUtilizationOfUnsecuredLines', 'age', 'DebtRatio', 'MonthlyIncome', 'NumberOfTimes90DaysLate']
df[cols].hist(bins=30, figsize=(12, 6), color='#4C78A8')
plt.suptitle('Distribuciones seleccionadas', y=1.02)
plt.tight_layout()
plt.show()

## Partición y pipeline
- Partición 70/15/15 con estratificación en cada paso.
- Pipeline: imputación (mediana) + StandardScaler aplicado a todas las variables numéricas.
Beneficio: se evita fuga de información y todos los modelos reciben datos comparables.

In [ ]:
X = df[features].copy()
y = df[target].astype(int).copy()

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=SEED, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=SEED, stratify=y_temp
)
print({'train': len(X_train), 'val': len(X_val), 'test': len(X_test)})

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])
preprocess = ColumnTransformer(transformers=[('num', numeric_transformer, features)])

def make_pipeline(model):
    return Pipeline([('prep', preprocess), ('model', model)])

models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=SEED),
}

## Validación cruzada (K=5)
Qué: entrenamos cada modelo, registramos AUC en train/val y aplicamos StratifiedKFold. Cómo interpretar: cv_mean_auc ≈ desempeño esperado; cv_std_auc ≈ varianza entre folds.

In [ ]:
import pandas as pd

def evaluate_model(name, estimator):
    estimator.fit(X_train, y_train)
    train_auc = roc_auc_score(y_train, estimator.predict_proba(X_train)[:, 1])
    val_auc = roc_auc_score(y_val, estimator.predict_proba(X_val)[:, 1])
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    cv_scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring='roc_auc')
    return {
        'model': name,
        'train_auc': train_auc,
        'val_auc': val_auc,
        'cv_mean_auc': float(cv_scores.mean()),
        'cv_std_auc': float(cv_scores.std(ddof=1)),
        'cv_scores': cv_scores,
    }

results = [evaluate_model(name, make_pipeline(model)) for name, model in models.items()]
res = pd.DataFrame(results).sort_values('cv_mean_auc', ascending=False)
res[['model', 'train_auc', 'val_auc', 'cv_mean_auc', 'cv_std_auc']]

## Selección y evaluación en prueba
Reentrenamos el mejor modelo con 	rain + val, medimos en 	est y graficamos curva ROC + matriz de confusión (umbral 0.5).

In [ ]:
best = res.iloc[0]['model']
best_estimator = make_pipeline(models[best])

X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

best_estimator.fit(X_trainval, y_trainval)
y_test_proba = best_estimator.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_test_proba)
print(f'Modelo seleccionado: {best}')
print(f'ROC AUC en prueba: {auc:.3f}')

fpr, tpr, _ = roc_curve(y_test, y_test_proba)
plt.plot(fpr, tpr, label=f'ROC (AUC={auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('FPR (1 - Especificidad)')
plt.ylabel('TPR (Sensibilidad)')
plt.title('Curva ROC en conjunto de prueba')
plt.legend()
plt.show()

ConfusionMatrixDisplay.from_estimator(best_estimator, X_test, y_test, cmap='Blues')
plt.title('Matriz de confusión (umbral 0.5)')
plt.show()

## Interpretaciones automáticas
Generamos texto con desbalance, faltantes y correlaciones principales.

In [ ]:
pos_rate = y.mean()
print(f'Proporción positiva (morosidad): {pos_rate:.2%}')
if pos_rate < 0.1:
    print('Desbalance severo: usar estratificación y métricas basadas en ranking.')
elif pos_rate < 0.3:
    print('Desbalance moderado: preferir ROC AUC y revisar umbrales.')
else:
    print('Clases relativamente balanceadas.')

na = miss[miss > 0]
if len(na):
    print('Columnas con faltantes (imputadas con mediana dentro del pipeline):')
    display(pd.DataFrame({'faltantes': na, '%': (na / len(df) * 100).round(2)}))
else:
    print('No se detectaron faltantes relevantes.')

corr = df[features + [target]].corr(numeric_only=True)[target].drop(target).sort_values(ascending=False)
print('Top correlaciones positivas con la morosidad:')
display(corr.head(5))
print('Top correlaciones negativas con la morosidad:')
display(corr.tail(5))

## Resumen en Markdown
Bloque autoexplicativo listo para pegar en informes o README.

In [ ]:
from IPython.display import Markdown, display
cv_mean = float(res[res['model'] == best]['cv_mean_auc'].values[0])
cv_std = float(res[res['model'] == best]['cv_std_auc'].values[0])
summary_md = f''' 
### Resumen de hallazgos
- Desbalance: proporción positiva = {pos_rate:.2%}. Estratificación + ROC AUC.
- Faltantes: {int(na.sum()) if len(na) else 0} valores en {len(na)} columnas (imputación mediana).
- Señales lineales: {', '.join(list(corr.head(3).index))} (positivas) y {', '.join(list(corr.tail(3).index))} (negativas).
- Mejor modelo: **{best}** con CV AUC = {cv_mean:.3f} (+/- {cv_std:.3f}).
- Prueba: AUC = {auc:.3f}; diferencia test − CV = {auc - cv_mean:+.3f}.
- Interpretación: alta utilización de crédito y múltiples atrasos elevan el riesgo; mayor edad/ingresos tienden a mitigarlo.
'''
display(Markdown(summary_md))

## Exportar resumen al README (opcional)
Inserta o actualiza una sección marcada en README.md.

In [ ]:
from datetime import datetime
start_marker = '<!-- AUTO-RESULTS-START -->'
end_marker = '<!-- AUTO-RESULTS-END -->'
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')
block = f"

{start_marker}
## Resultados (autogenerado)

Generado desde Trabajo2.ipynb el {timestamp}.

{summary_md}
{end_marker}
"

readme_path = Path('README.md')
if readme_path.exists():
    txt = readme_path.read_text(encoding='utf-8', errors='ignore')
    if start_marker in txt and end_marker in txt:
        before = txt.split(start_marker)[0]
        after = txt.split(end_marker)[-1]
        new_text = before + block + after
    else:
        new_text = txt + block
else:
    new_text = '# README' + block

readme_path.write_text(new_text, encoding='utf-8')
print('Resumen de hallazgos escrito en README.md')

## Conclusiones y próximos pasos
- Reportar el AUC de prueba junto con la media y desviación de CV.
- Ajustar umbrales según costos y analizar la curva Precision‑Recall.
- Explorar búsqueda de hiperparámetros o modelos ensemble y técnicas para lidiar con el desbalance (class_weight, SMOTE, undersampling).
- Traducir las señales (utilización, atrasos) en políticas de crédito accionables.